In [21]:
# MAKE SURE YOU HAVE THE DATA SET DOWNLOADED
import cv2
import os
import numpy as np

In [24]:
# location of extracted images
image_dir = "./final_v1" 
save_pth = "./heatmap"

show=True
save=True

# The area to process 
# [[Top left xy][Bottom right xy]],
# (default [[0, 0][999, 999]])
area = [
    [0, 0], 
    [999, 999]
]

# Intensity: how intense each pixel change is (scale goes from 0-255)
# Decay: how much decay between each frame 
intensity = 140
decay = 10

In [23]:
# Helper function to resize small images

def ResizeWithAspectRatio(image, width=None, height=None, inter=cv2.INTER_AREA):
    dim = None
    (h, w) = image.shape[:2]

    if width is None and height is None:
        return image
    if width is None:
        r = height / float(h)
        dim = (int(w * r), height)
    else:
        r = width / float(w)
        dim = (width, int(h * r))

    return cv2.resize(image, dim, interpolation=inter)

In [25]:
dirs = [x[0] for x in os.walk(image_dir)]
# print(dirs)
img_paths = []
for folder in dirs: 
     for file in os.listdir(folder):
          filename = os.fsdecode(file)
          if filename.endswith("png"): 
               img_paths.append(os.path.join(f"{image_dir}/{folder}/", filename))

img_paths.sort(key = lambda x: x.split("/")[-1][:-4])

In [6]:
# Cell to get location of specific points on the image (useful for the area config param)

def onMouse(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
       print('x = %d, y = %d'%(x, y))

cv2.imshow("image", cv2.imread(img_paths[1000]))
cv2.namedWindow('image')
cv2.setMouseCallback('image', onMouse)
cv2.waitKey(0)


32

In [75]:
img3 = np.zeros((area[1][1]-area[0][1], area[1][0]-area[0][0], 3))

if not os.path.isdir(save_pth):
    os.mkdir(save_pth)
    os.mkdir(save_pth + "/0")
    os.mkdir(save_pth + "/1")
    os.mkdir(save_pth + "/2")
    os.mkdir(save_pth + "/3")

for i, path in enumerate(img_paths[1:]):
    print(f"{i} / {len(img_paths)}", end="\r")
    img = cv2.imread(img_paths[i])
    img2 = cv2.imread(path)
    
    if img_paths[i+1].split("/")[-1][0] != img_paths[i].split("/")[-1][0]: 
        continue

    img = img[area[0][1]:area[1][1], area[0][0]:area[1][0]]
    img2 = img2[area[0][1]:area[1][1], area[0][0]:area[1][0]]
 
    # Sequential frame colour masks
    # (Detects changes for each pixel between sequential images)
    b_1, g_1, r_1 = img[..., 0], img[..., 1], img[..., 2]
    b_2, g_2, r_2 = img2[..., 0], img2[..., 1], img2[..., 2]
    mask = (b_1 == b_2) & (g_1 == g_2) & (r_1 == r_2)
    mask = ~mask

    b, g, r = img3[..., 0], img3[..., 1], img3[..., 2]
    r_mask = (r < 1.0)
    g_mask = (~r_mask) & (g < 1.0)
    b_mask = (~r_mask) & (~g_mask) & (b < 1.0)

    # If changes detected increase the brightness 
    # (Using "Hot" colour map)
    # Work around for numpy masking issues (probably a faster way of doing this)
    temp_image = img3[mask & r_mask]
    temp_image[:, 2] += intensity / 255
    temp_image[:, 1] += (temp_image[:, 2] - 1).clip(0, 1)
    img3[mask & r_mask] = temp_image


    temp_image = img3[mask & g_mask]
    temp_image[:, 1] += intensity / 255
    temp_image[:, 0] += (temp_image[:, 1] - 1).clip(0, 1)
    img3[mask & g_mask] = temp_image


    temp_image = img3[mask & b_mask]
    temp_image[:, 0] += intensity / 255
    img3[mask & b_mask] = temp_image

    img3 = img3.clip(0.0, 1.0)
    


    if img3.shape[1] < 300:
        if save: 
            img3 = ResizeWithAspectRatio(img3)
            alpha = np.sum(img3, axis=-1) > 0
            cv2.imwrite(f"{save_pth}/{tile}-{i}.exr", np.dstack((img3, alpha)).astype("float32"))
        if show: 
            cv2.imshow("Heatmap", ResizeWithAspectRatio(img3.astype("float32")))
            cv2.waitKey(1)
    else:
        if show: 
            

            cv2.imshow(f"Heatmap", img3)
            cv2.waitKey(1)
        if save: 
            tile = img_paths[i].split("/")[-1][0]
            file_name = img_paths[i].split("/")[-1][:-4]
            cv2.imwrite(f"{save_pth}/{tile}/{file_name}.exr", img3.astype("float32"))
            

   
    b, g, r = img3[..., 0], img3[..., 1], img3[..., 2]
    r_mask = (r > 0) & (g == 0.0) & (b == 0.0)
    g_mask =  (g > 0.0) & (b == 0.0)
    b_mask = (b > 0.0)

    # Decay hotspots quicker  
    temp_image = img3[b_mask]
    temp_image[:, 0] -= decay / 255
    img3[b_mask] = temp_image
    temp_image = img3[g_mask]
    temp_image[:, 1] -= decay / 255
    img3[g_mask] = temp_image
    temp_image = img3[r_mask]
    temp_image[:, 2] -= decay / 255
    img3[r_mask] = temp_image